### Programming Assignment 3 
# **Logicstic Regression**

AJ Dave A. Obis and Juan Francis C. Tuatis

In [3]:
# Import necessary libraries
import pandas as pd

# Load the data
# Specify the column names based on the wdbc.names file
column_names = [
    'Sample code number', 'Clump Thickness', 'Uniformity of Cell Size',
    'Uniformity of Cell Shape', 'Marginal Adhesion', 'Single Epith. Cell Size',
    'Bare Nuclei', 'Bland Chromatin', 'Normal Nucleoli', 'Mitoses', 'Class'
]

# Read the data into a DataFrame
data = pd.read_csv('wdbc.data', header=None, names=column_names)

### Data Pre-Processing

In [4]:
# Remove rows with missing values
data.dropna(inplace=True)

# Remove the column "Sample code number"
data.drop(columns=['Sample code number'], inplace=True)

# Replace "Class" values: 0 for benign, 1 for malignant
data['Class'] = data['Class'].map({'B': 0, 'M': 1})

# Display the first few rows of the processed DataFrame
print(data.head())

                                                                                                                                                        Clump Thickness  \
842302   M 17.99 10.38 122.80 1001.0 0.11840 0.27760 0.3001 0.14710 0.2419 0.07871 1.0950 0.9053 8.589 153.40 0.006399 0.04904 0.05373 0.01587 0.03003            25.38   
842517   M 20.57 17.77 132.90 1326.0 0.08474 0.07864 0.0869 0.07017 0.1812 0.05667 0.5435 0.7339 3.398 74.08  0.005225 0.01308 0.01860 0.01340 0.01389            24.99   
84300903 M 19.69 21.25 130.00 1203.0 0.10960 0.15990 0.1974 0.12790 0.2069 0.05999 0.7456 0.7869 4.585 94.03  0.006150 0.04006 0.03832 0.02058 0.02250            23.57   
84348301 M 11.42 20.38 77.58  386.1  0.14250 0.28390 0.2414 0.10520 0.2597 0.09744 0.4956 1.1560 3.445 27.23  0.009110 0.07458 0.05661 0.01867 0.05963            14.91   
84358402 M 20.29 14.34 135.10 1297.0 0.10030 0.13280 0.1980 0.10430 0.1809 0.05883 0.7572 0.7813 5.438 94.44  0.011490 0.02461 0.05688 0.01885 0.